In [33]:
%load_ext autoreload
%autoreload 2
from qm.QuantumMachinesManager import QuantumMachinesManager
from qm.qua import *
from qm import SimulationConfig
import numpy as np
import matplotlib.pyplot as plt
config = {
    "version": 1,
    "controllers": {
        "con1": {
            "type": "opx1",
            "analog_outputs": {
                1: {"offset": +0.0},
                2: {"offset": +0.0},
            },
        }
    },
    "elements": {
        "qe1": {
            "singleInput": {"port": ("con1", 1)},
            "intermediate_frequency": 5e6,
            "operations": {
                "playOp": "constPulse",
            },
        },
        "qe2": {
            "singleInput": {"port": ("con1", 2)},
            "intermediate_frequency": 5e6,
            "operations": {
                "playOp": "constPulse",
            },
        },
    },
    "pulses": {
        "constPulse": {
            "operation": "control",
            "length": 1000,  # in ns
            "waveforms": {"single": "const_wf"},
        },
    },
    "waveforms": {
        "const_wf": {"type": "constant", "sample": 0.2},
    },
}


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# QUA variables and math

The QUA langaugage has variables of three data types: `int`egers, `fixed` point numbers and `booleans`. You can also have vectors of such types (but not of mixed types). These variables can be populated by mathematical operations or measurements. There are also reserved varibales you can write directly to from python, but this will not be covered in this demo.

The `fixed` point data type may be less familiar to some. It's a way to represent decimal numbers on hardware. Because there is a finite amount or memory allocated for each number, you need to decide on a way to represent the integer part and the decimal part of this number. On a personal computer, this is done with a floating-point representation which is able to decide how many bits are allocated for the integer part and how many are for the decimal (hence -floating point). This is computationally intensive so, for the time being, we have a fixed representation. We allocate 4 bits for the integer part and 28 for the decimal, giving a number range between -2 and 2 (in fact, slighly less so try not to get to 2 exactly).

QUA's real time mathematics capabilities are extensive. We include arithmatic operations, trigonometry, vector operations, random number generation and more. Read about it in the [docs](https://qm-docs.s3.amazonaws.com/v0.8/python/qua_ref.html?highlight=math).

The following example show the simplest possible example: storing a number to a variable and then saving it to be read outside the program.

In [16]:
with program() as prog:
    a = declare(int,value=10)
    save(a,"a")

QMm = QuantumMachinesManager()
QMm.close_all_quantum_machines()

QM1 = QMm.open_qm(config)
job = QM1.simulate(prog, SimulationConfig(int(1000)))  # in clock cycles, 4 ns
res=job.result_handles

a = res.a.fetch_all()['value']

print(f"\nThe program returned the value: a={a}")

    

2021-03-30 17:53:06,446 - qm - INFO - Performing health check
2021-03-30 17:53:06,450 - qm - INFO - Health check passed
2021-03-30 17:53:06,453 - qm - WARNING - Client's version (0.90.690) does not match the server version (0.91.781).
2021-03-30 17:53:06,464 - qm - INFO - to simulate a program, use QuantumMachinesManager.simulate(..)
2021-03-30 17:53:06,471 - qm - INFO - Flags: 
2021-03-30 17:53:06,471 - qm - INFO - Simulating Qua program

The program returned the value: a=[10]


In [19]:
with program() as prog:
    a = declare(int,value=10)
    assign(a,2*(a+1))
    save(a,"a")

QMm = QuantumMachinesManager()
QMm.close_all_quantum_machines()

QM1 = QMm.open_qm(config)
job = QM1.simulate(prog, SimulationConfig(int(1000)))  # in clock cycles, 4 ns
res=job.result_handles

a = res.a.fetch_all()['value']

print(f"\nThe program returned the value: a={a}")

    

2021-03-30 17:54:49,672 - qm - INFO - Performing health check
2021-03-30 17:54:49,675 - qm - INFO - Health check passed
2021-03-30 17:54:49,678 - qm - WARNING - Client's version (0.90.690) does not match the server version (0.91.781).
2021-03-30 17:54:49,687 - qm - INFO - to simulate a program, use QuantumMachinesManager.simulate(..)
2021-03-30 17:54:49,695 - qm - INFO - Flags: 
2021-03-30 17:54:49,696 - qm - INFO - Simulating Qua program

The program returned the value: a=[22]


In [24]:
with program() as prog:
    a = declare(fixed)
    assign(a,Math.cos2pi(0.5))
    save(a,"a")

QMm = QuantumMachinesManager()
QMm.close_all_quantum_machines()

QM1 = QMm.open_qm(config)
job = QM1.simulate(prog, SimulationConfig(int(1000)))  # in clock cycles, 4 ns
res=job.result_handles

a = res.a.fetch_all()['value']

print(f"\nThe program returned the value: a={a}")


2021-03-30 17:59:29,084 - qm - INFO - Performing health check
2021-03-30 17:59:29,088 - qm - INFO - Health check passed
2021-03-30 17:59:29,092 - qm - WARNING - Client's version (0.90.690) does not match the server version (0.91.781).
2021-03-30 17:59:29,102 - qm - INFO - to simulate a program, use QuantumMachinesManager.simulate(..)
2021-03-30 17:59:29,109 - qm - INFO - Flags: 
2021-03-30 17:59:29,110 - qm - INFO - Simulating Qua program

The program returned the value: a=[-1.]


In [31]:
with program() as prog:
    r = Random()
    a = declare(int)
    b = declare(fixed)
    assign(a, r.rand_int(100)) # a will be a number between 0 and 99
    assign(b, r.rand_fixed()) # b will be a number between 0.0 and 1.0

    save(a,"a")
    save(b,"b")

QMm = QuantumMachinesManager()
QMm.close_all_quantum_machines()

QM1 = QMm.open_qm(config)
job = QM1.simulate(prog, SimulationConfig(int(1000)))  # in clock cycles, 4 ns
res=job.result_handles

a = res.a.fetch_all()['value']
b = res.b.fetch_all()['value']

print(f"\nThe program returned the value: a={a}")
print(f"\nThe program returned the value: b={b}")

2021-03-30 18:00:59,518 - qm - INFO - Performing health check
2021-03-30 18:00:59,521 - qm - INFO - Health check passed
2021-03-30 18:00:59,523 - qm - WARNING - Client's version (0.90.690) does not match the server version (0.91.781).
2021-03-30 18:00:59,532 - qm - INFO - to simulate a program, use QuantumMachinesManager.simulate(..)
2021-03-30 18:00:59,538 - qm - INFO - Flags: 
2021-03-30 18:00:59,539 - qm - INFO - Simulating Qua program

The program returned the value: a=[72]

The program returned the value: b=[0.42404462]
